In [1]:
import pandas as pd 
import requests 

df = pd.read_csv('erie_total.csv')

df

,Name,Population,Sq Mi,Zip Code,6/9/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,...,7/17/2020,7/18/2020,7/19/2020,7/20/2020,7/23/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,7/29/2020
0,Akron,9549,59.61,14001,34,35,36,37,37,38,...,39,39,39,39,39,39,39,39,39,39
1,Alden,12851,46.68,14004,61,61,61,61,61,61,...,69,69,69,70,70,71,71,72,72,72
2,Angola,10113,28.86,14006,17,17,17,17,17,17,...,29,29,29,30,30,30,30,30,30,30
3,Basom,1923,41.17,14013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boston,3060,23.54,14025,9,9,9,9,9,9,...,11,11,11,11,11,11,11,12,12,12
5,Bowmansville,910,0.79,14026,3,3,3,3,3,3,...,4,4,4,4,4,4,4,4,4,4
6,Chaffee,1867,20.90,14030,1,1,1,2,2,2,...,2,2,2,2,2,2,2,2,2,2
7,Clarence,10145,20.83,14031,76,76,76,78,78,78,...,84,84,84,84,84,84,84,84,84,84
8,Clarence Center,8964,25.14,14032,35,35,35,35,35,35,...,45,47,47,47,48,50,50,50,50,50
9,Colden,2396,19.96,14033,1,1,1,1,1,1,...,3,3,3,3,3,3,3,3,3,3


In [4]:
#Access website date

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:78.0) Gecko/20100101 Firefox/78.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Origin': 'http://erieny.maps.arcgis.com',
    'Connection': 'keep-alive',
    'Referer': 'http://erieny.maps.arcgis.com/apps/opsdashboard/index.html',
    'TE': 'Trailers',
}

params1 = (
    ('f', 'json'),
)

response = requests.get('https://erieny.maps.arcgis.com/sharing/rest/content/items/dd7f1c0c352e4192ab162a1dfadc58e1/data', headers=headers, params=params1)


#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://erieny.maps.arcgis.com/sharing/rest/content/items/dd7f1c0c352e4192ab162a1dfadc58e1/data?f=json', headers=headers)

timestamp = response.json()['headerPanel']['subtitle']


#return date
date = timestamp[9:18]

time = timestamp[20:-1]

print('Date: '+str(date))
print('Time: '+str(time))
#--------------
#import daily cases
params = (
    ('f', 'json'),
    ('where', '1=1'),
    ('returnGeometry', 'false'),
    ('spatialRel', 'esriSpatialRelIntersects'),
    ('outFields', '*'),
    ('orderByFields', 'ZIP_CODE asc'),
    ('resultOffset', '0'),
    ('resultRecordCount', '80'),
    ('resultType', 'standard'),
    ('cacheHint', 'true'),
)

response = requests.get('https://services1.arcgis.com/CgOSc11uky3egK6O/arcgis/rest/services/erie_zip_codes_confirmed_counts/FeatureServer/0/query', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://services1.arcgis.com/CgOSc11uky3egK6O/arcgis/rest/services/erie_zip_codes_confirmed_counts/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=ZIP_CODE%20asc&resultOffset=0&resultRecordCount=80&resultType=standard&cacheHint=true', headers=headers)



data = response.json()['features']
#explore these!!

lists = []
for a in data:
    zipcode = a['attributes']['ZIP_CODE']
    confirmed =a['attributes']['CONFIRMED']
    #pop =a['attributes']['POPULATION']
    #name = a['attributes']['PO_NAME']
    
    lists.append((zipcode,confirmed))
    
df_today = pd.DataFrame(lists,columns=['Zip Code',date])

#return df_today

new_data = df_today.columns[-1]
prev = df.columns[-1]

print('prev: '+str(prev))
print('new: '+str(new_data))

if new_data > prev:
    df[date] = df_today[date]
    df.to_csv('erie_total.csv',index=False)
    print('updated...')
else:
    print('Already updated')

Date: 7/29/2020
Time: 5PM EST
prev: 7/29/2020
new: 7/29/2020
Already updated


In [5]:
df_confirm = pd.read_csv('erie_total.csv')

df_confirm.head()

,Name,Population,Sq Mi,Zip Code,6/9/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,...,7/17/2020,7/18/2020,7/19/2020,7/20/2020,7/23/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,7/29/2020
0,Akron,9549,59.61,14001,34,35,36,37,37,38,...,39,39,39,39,39,39,39,39,39,39
1,Alden,12851,46.68,14004,61,61,61,61,61,61,...,69,69,69,70,70,71,71,72,72,72
2,Angola,10113,28.86,14006,17,17,17,17,17,17,...,29,29,29,30,30,30,30,30,30,30
3,Basom,1923,41.17,14013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boston,3060,23.54,14025,9,9,9,9,9,9,...,11,11,11,11,11,11,11,12,12,12
